Import Python libraries. 

In [58]:
import numpy as np
from urllib.request import urlopen
import urllib
import matplotlib.pyplot as plt # Visuals
import seaborn as sns 
import sklearn as skl
import pandas as pd

from sklearn.cross_validation import train_test_split # Create training and test sets
from sklearn.neighbors import KNeighborsClassifier # Kth Nearest Neighbor
from sklearn.tree import DecisionTreeClassifier # Decision Trees
from sklearn.tree import export_graphviz # Extract Decision Tree visual
from sklearn.ensemble import RandomForestClassifier # Random Forest
from sklearn.metrics import roc_curve # ROC Curves
from sklearn.metrics import auc # AUC 
from urllib.request import urlopen # Get data from UCI Machine Learning Repository

Begin import of data into the notebook. Because we are working with three different datasets, we had to merge the data into one array. 

print(data) displays the entire dataframe. 

In [48]:
Cleveland_data_URL = 'http://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.hungarian.data'
Hungarian_data_URL = 'http://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.hungarian.data'
Switzerland_data_URL = 'http://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.switzerland.data'
np.set_printoptions(threshold=np.nan)#makes it so that we can see a whole array when we output it

names = ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'heart disease(by angiography)']
ClevelandHeartDisease = pd.read_csv(urlopen(Cleveland_data_URL), names = names)#gets Cleveland data
HungarianHeartDisease = pd.read_csv(urlopen(Hungarian_data_URL), names = names)#gets Hungary data
SwitzerlandHeartDisease = pd.read_csv(urlopen(Switzerland_data_URL), names = names)#gets Switzerland data
datatemp = [ClevelandHeartDisease, HungarianHeartDisease, SwitzerlandHeartDisease]#combines all arrays into a list
#heartDisease.set_index(['id_number'], inplace = True)
#I don't know what that did, but it was from Raoul's code and it was giving me bugs so I got rid of it
heartDisease = pd.concat(datatemp)#combines list into one array


We will begin the exploratory analysis of our dataset. 

We don't want to predict on all the variables so we are getting rid of 'ca', 'slope', and 'thal'.
For the variables we kept, there are still some "?" in the data, so we're going to replace them with a NAN. 

In [49]:
del heartDisease['ca']
del heartDisease['slope']
del heartDisease['thal']

heartDisease = heartDisease.replace('?', np.nan)
heartDisease.dtypes

age                               int64
sex                               int64
cp                                int64
trestbps                         object
chol                             object
fbs                              object
restecg                          object
thalach                          object
exang                            object
oldpeak                          object
heart disease(by angiography)     int64
dtype: object

In [52]:
#loop to count the "?" per variable. More for interpretation but we don't really use it. 

count = 0
for item in heartDisease:
    for i in heartDisease[item]:
        count += (i == '?')

Now we can begin normalizing the data. First the data are all converted to float items and the function "normalize" does this!!!! 

In [37]:
for item in heartDisease: #converts everything to floats
    heartDisease[item] = pd.to_numeric(heartDisease[item])

def normalize(heartDisease, toNormalize): #normalizes 
    result = heartDisease.copy()
    for item in heartDisease.columns:
        if (item in toNormalize):
            max_value = heartDisease[item].max()
            min_value = heartDisease[item].min()
            result[item] = (heartDisease[item] - min_value) / (max_value - min_value)
    return result
toNormalize = ['age', 'cp', 'trestbps', 'chol', 'thalach', 'oldpeak'] #columns to normalize
heartDisease = normalize(heartDisease, toNormalize)
heartDisease = heartDisease.dropna()
heartDisease.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,heart disease(by angiography)
0,0.000000,1,0.333333,0.416667,0.218905,0.0,2.0,0.961538,0.0,0.342105,0
1,0.021739,1,0.333333,0.333333,0.402985,0.0,0.0,0.769231,0.0,0.342105,0
3,0.043478,0,0.000000,0.750000,0.393035,0.0,1.0,0.846154,0.0,0.342105,0
4,0.065217,0,0.333333,0.166667,0.363184,0.0,1.0,0.692308,0.0,0.342105,0
5,0.086957,0,0.333333,0.208333,0.328358,0.0,0.0,0.807692,0.0,0.342105,0


This is a classification problem, so to simplify our project we are going to convert the predictor column into 1 for "heart disease is present" and 0 for "heart disease is not present."

Before, the scope of the disease ran from 0 - 5 for the intensity of the heart disease but this shit's too hard so we're going to replace it.

In [53]:
for i in range(1,5):
    heartDisease['heart disease(by angiography)'] = heartDisease['heart disease(by angiography)'].replace(i,1)
#heartDisease

VISUALIZATION TIME

In [59]:
f, ax = plt.subplots(figsize=(11, 15))

ax.set_axis_bgcolor('#fafafa')
plt.title("Box Plot of Transformed Data Set")
ax.set(xlim=(-.05, 1.05))
ax = sns.boxplot(data = heartDisease[1:29], orient = 'h', palette = 'Set2')

Whoooo!!! Let's play with some algorithms. First, split the data into training and test sets. 

In [46]:
train, test = train_test_split(heartDisease, test_size = 0.20, random_state = 42)